## makemore - MLP v2, closer look

This implementation includes:
- Multiple layers
- Diagnostics and visualizations
- BatchNorm

In [3]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

import random
import numpy as np

names = open('data/names.txt', 'r').read().splitlines()
print("Sample names: ", names[5:12])
print("Number of names: ", len(names))

# create char to index mapping for set (vocabulary) of chars in names
chars = sorted(list(set(''.join(names))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print("Character mapping: ", itos)
print("Vocab Size: ", vocab_size)

Sample names:  ['charlotte', 'mia', 'amelia', 'harper', 'evelyn', 'abigail', 'emily']
Number of names:  32033
Character mapping:  {1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
Vocab Size:  27


In [ ]:
def build_dataset(names):
    context_length = 3 
    X, Y = [], []
    for name in names:

        context = [0] * context_length 
        for char in name + '.':
            ix = stoi[char]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X,Y

random.seed(1337)
random.shuffle(names)
n1 = int(0.8*len(names))
n2 = int(0.9*len(names))

Xtrain, Ytrain = build_dataset(names[:n1])  # 80%
Xval, Yval = build_dataset(names[n1:n2])    # 10%
Xtest, Ytest = build_dataset(names[n2:])    # 10%

torch.Size([182594, 3]) torch.Size([182594])
torch.Size([22690, 3]) torch.Size([22690])
torch.Size([22864, 3]) torch.Size([22864])


In [ ]:
dim_embed = 10
hidden_layer_size = 200
context_length = 3

